In [1]:
import torch
import torch.nn as nn
from transformers import SegformerConfig, SegformerForSemanticSegmentation
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
from tqdm import tqdm
import h5py
from sklearn.model_selection import train_test_split
import cv2
import time

# ======================
# 1. FIXED SegFormer Model with Upsampling
# ======================

class SegFormerLandslide128(nn.Module):
    def __init__(self, num_classes=1, input_size=128, output_size=128):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        # Load configuration and modify for 128 channels and num_classes
        config = SegformerConfig.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
        config.num_channels = 128  # CRITICAL: Change from 3 to 128
        config.num_labels = num_classes  # Set number of output classes
        
        # Load model with modified configuration
        self.segformer = SegformerForSemanticSegmentation.from_pretrained(
            "nvidia/segformer-b0-finetuned-ade-512-512",
            config=config,
            ignore_mismatched_sizes=True
        )
        
        # Add upsampling layer to match output size
        self.upsample = nn.Upsample(size=output_size, mode='bilinear', align_corners=True)
        
        print("✓ SegFormer initialized for 128 input channels with upsampling")
    
    def forward(self, x):
        # x shape: [batch_size, 128, height, width]
        outputs = self.segformer(x).logits  # [B, 1, 32, 32] for input [B, 128, 128, 128]
        return self.upsample(outputs)  # [B, 1, 128, 128]

# ======================
# 2. Updated Loss Function with Size Handling
# ======================

class LandslideLoss(nn.Module):
    def __init__(self, alpha=0.7):
        super().__init__()
        self.alpha = alpha
        self.bce = nn.BCEWithLogitsLoss()
    
    def dice_loss(self, pred, target):
        smooth = 1.0
        pred = torch.sigmoid(pred)
        
        # Ensure same spatial dimensions
        if pred.shape[-2:] != target.shape[-2:]:
            pred = F.interpolate(pred, size=target.shape[-2:], mode='bilinear', align_corners=True)
            
        intersection = (pred * target).sum()
        union = pred.sum() + target.sum()
        return 1.0 - (2.0 * intersection + smooth) / (union + smooth)
    
    def forward(self, pred, target):
        # Ensure pred and target have same spatial dimensions
        if pred.shape[-2:] != target.shape[-2:]:
            pred = F.interpolate(pred, size=target.shape[-2:], mode='bilinear', align_corners=True)
            
        bce_loss = self.bce(pred, target)
        dice_loss = self.dice_loss(pred, target)
        return self.alpha * bce_loss + (1 - self.alpha) * dice_loss

def calculate_iou(pred, target):
    pred = (torch.sigmoid(pred) > 0.5).float()
    
    # Ensure same spatial dimensions
    if pred.shape[-2:] != target.shape[-2:]:
        pred = F.interpolate(pred, size=target.shape[-2:], mode='bilinear', align_corners=True)
        
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + 1e-6) / (union + 1e-6)

# ======================
# 3. Dataset Class (Same as before)
# ======================

class OptimizedLandslideH5Dataset128(Dataset):
    def __init__(self, img_h5_paths, mask_h5_paths=None, img_size=128, is_train=True, max_samples=None):
        self.img_h5_paths = img_h5_paths
        self.mask_h5_paths = mask_h5_paths
        self.img_size = img_size
        self.is_train = is_train
        
        # We'll handle transforms manually for 128-channel data
        self.is_train = is_train
        
        # PRELOAD all file indices for maximum speed
        self.total_samples = 0
        self.file_indices = []
        
        print("Preloading file indices for 128-channel data...")
        for file_idx, img_path in enumerate(tqdm(img_h5_paths, desc="Scanning HDF5 files")):
            try:
                with h5py.File(img_path, 'r') as img_file:
                    img_key = list(img_file.keys())[0]
                    n_samples = img_file[img_key].shape[0]
                    
                    for sample_idx in range(n_samples):
                        if max_samples and self.total_samples >= max_samples:
                            break
                        self.file_indices.append((file_idx, sample_idx))
                        self.total_samples += 1
                    
                    if max_samples and self.total_samples >= max_samples:
                        break
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                continue
        
        print(f"✓ Loaded {self.total_samples} samples with 128 channels")
    
    def __len__(self):
        return self.total_samples
    
    def _apply_transforms(self, image, mask):
        """Manual transform for 128-channel data"""
        # Resize
        if image.shape[0] != self.img_size or image.shape[1] != self.img_size:
            # Resize each channel separately
            resized_image = np.zeros((self.img_size, self.img_size, image.shape[2]), dtype=np.float32)
            for c in range(image.shape[2]):
                resized_image[:, :, c] = cv2.resize(image[:, :, c], (self.img_size, self.img_size), interpolation=cv2.INTER_LINEAR)
            image = resized_image
        
        if mask.shape[0] != self.img_size or mask.shape[1] != self.img_size:
            mask = cv2.resize(mask, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)
        
        # Data augmentation for training
        if self.is_train:
            # Horizontal flip
            if np.random.random() > 0.5:
                image = np.fliplr(image).copy()
                mask = np.fliplr(mask).copy()
            
            # Vertical flip
            if np.random.random() > 0.5:
                image = np.flipud(image).copy()
                mask = np.flipud(mask).copy()
        
        # Convert to PyTorch format: (channels, height, width)
        image = torch.from_numpy(image).float().permute(2, 0, 1)
        mask = torch.from_numpy(mask).float().unsqueeze(0)
        
        return image, mask
    
    def __getitem__(self, idx):
        file_idx, sample_idx = self.file_indices[idx]
        img_path = self.img_h5_paths[file_idx]
        mask_path = self.mask_h5_paths[file_idx] if self.mask_h5_paths else None
        
        try:
            # Load 128-channel image
            with h5py.File(img_path, 'r') as img_file:
                img_key = list(img_file.keys())[0]
                img = img_file[img_key][sample_idx]  # Shape could be (128, 128, 128) or (128, 128)
                
                # Handle different possible shapes
                if len(img.shape) == 2:
                    # Single channel - repeat to 128 channels (shouldn't happen but safe)
                    img = np.stack([img] * 128, axis=-1)
                elif len(img.shape) == 3:
                    # If shape is (128, 128, 128) - perfect
                    if img.shape[0] == 128 and img.shape[2] == 128:
                        # Shape is (128, height, width) - channels first, convert to channels last
                        img = np.transpose(img, (1, 2, 0))
                
                # Ensure we have exactly 128 channels
                if img.shape[-1] > 128:
                    img = img[:, :, :128]
                elif img.shape[-1] < 128:
                    # Pad with zeros if fewer channels
                    padding = np.zeros((img.shape[0], img.shape[1], 128 - img.shape[-1]), dtype=img.dtype)
                    img = np.concatenate([img, padding], axis=-1)
                
                # Convert to float32 and normalize
                img = img.astype(np.float32)
                
                # Normalize each channel separately
                for c in range(img.shape[-1]):
                    channel_data = img[:, :, c]
                    if np.std(channel_data) > 0:
                        img[:, :, c] = (channel_data - np.mean(channel_data)) / np.std(channel_data)
                
                # Load mask
                mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)
                if mask_path and os.path.exists(mask_path):
                    with h5py.File(mask_path, 'r') as mask_file:
                        mask_key = list(mask_file.keys())[0]
                        mask_data = mask_file[mask_key]
                        
                        if len(mask_data.shape) == 3 and sample_idx < mask_data.shape[0]:
                            mask = mask_data[sample_idx]
                        elif len(mask_data.shape) == 2:
                            mask = mask_data[:]
                        
                        if len(mask.shape) == 3:
                            mask = mask.squeeze()
                        mask = (mask > 0).astype(np.float32)
                
                # Apply transforms
                image_tensor, mask_tensor = self._apply_transforms(img, mask)
                
                return {
                    'image': image_tensor,  # Shape: [128, 128, 128]
                    'mask': mask_tensor    # Shape: [1, 128, 128]
                }
                
        except Exception as e:
            print(f"Error loading sample {idx}: {e}")
            # Fast fallback - zero tensor with 128 channels
            dummy_img = torch.zeros(128, self.img_size, self.img_size)
            dummy_mask = torch.zeros(1, self.img_size, self.img_size)
            return {'image': dummy_img, 'mask': dummy_mask}

# ======================
# 4. Updated Training Function with Modern AMP
# ======================

def optimized_train_model_128(model, train_loader, val_loader, epochs=50, device='cuda'):
    model = model.to(device)
    
    # Modern mixed precision for RTX 4070
    scaler = torch.amp.GradScaler(device)  # Updated for modern PyTorch
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    criterion = LandslideLoss(alpha=0.7)
    
    best_val_iou = 0.0
    
    print("Starting optimized training for 128-channel data with mixed precision...")
    
    for epoch in range(epochs):
        epoch_start = time.time()
        
        # Training with mixed precision
        model.train()
        train_loss = 0
        train_iou = 0
        
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
        
        for batch_idx, batch in enumerate(train_bar):
            images = batch['image'].to(device, non_blocking=True)
            masks = batch['mask'].to(device, non_blocking=True)
            
            # Debug: Print shapes on first batch
            if batch_idx == 0 and epoch == 0:
                print(f"Input image shape: {images.shape}")  # Should be [B, 128, H, W]
                print(f"Input mask shape: {masks.shape}")    # Should be [B, 1, H, W]
                print(f"Image range: [{images.min():.3f}, {images.max():.3f}]")
                print(f"Mask unique values: {torch.unique(masks)}")
            
            optimizer.zero_grad()
            
            # Modern mixed precision forward
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(images)
                loss = criterion(outputs, masks)
            
            # Mixed precision backward
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            train_iou += calculate_iou(outputs, masks).item()
            
            if batch_idx % 50 == 0:
                train_bar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'avg_loss': f'{train_loss/(batch_idx+1):.4f}',
                    'GPU_mem': f'{torch.cuda.memory_allocated()/1024**3:.2f}GB'
                })
        
        # Validation
        model.eval()
        val_loss = 0
        val_iou = 0
        
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device, non_blocking=True)
                masks = batch['mask'].to(device, non_blocking=True)
                
                with torch.amp.autocast(device_type='cuda'):
                    outputs = model(images)
                    loss = criterion(outputs, masks)
                
                val_loss += loss.item()
                val_iou += calculate_iou(outputs, masks).item()
        
        epoch_time = time.time() - epoch_start
        
        avg_train_loss = train_loss / len(train_loader)
        avg_train_iou = train_iou / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        avg_val_iou = val_iou / len(val_loader)
        
        scheduler.step()
        
        print(f'\nEpoch {epoch+1}/{epochs} ({epoch_time/60:.1f} min):')
        print(f'  Train Loss: {avg_train_loss:.4f}, Train IoU: {avg_train_iou:.4f}')
        print(f'  Val Loss: {avg_val_loss:.4f}, Val IoU: {avg_val_iou:.4f}')
        print(f'  LR: {scheduler.get_last_lr()[0]:.2e}')
        print(f'  GPU Memory: {torch.cuda.memory_allocated()/1024**3:.2f}GB')
        
        if avg_val_iou > best_val_iou:
            best_val_iou = avg_val_iou
            torch.save(model.state_dict(), 'best_landslide_model_128ch.pth')
            print(f'  ✓ Saved best model (val_iou: {avg_val_iou:.4f})')

# ======================
# 5. ULTRA-FAST TRAINING for 128-Channel Data
# ======================

def ultra_fast_training_128():
    """Ultra-fast training for 128-channel data"""
    device = torch.device('cuda')
    print(f"🚀 Starting ULTRA-FAST training for 128-channel data on RTX 4070...")
    
    base_path = "/home/neel/Geog_project/Landslide4Sense_1"
    
    def find_h5_files(directory):
        if not os.path.exists(directory):
            return []
        return sorted([os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.h5')])
    
    train_img_files = find_h5_files(os.path.join(base_path, "TrainData", "img"))
    train_mask_files = find_h5_files(os.path.join(base_path, "TrainData", "mask"))
    
    # Adjusted for 128 channels - smaller batch size due to increased memory
    IMG_SIZE = 128
    BATCH_SIZE = 16  # Reduced from 24 due to 128 channels
    SUBSET_SIZE = 50000  # Reduced due to memory constraints
    NUM_WORKERS = min(os.cpu_count(), 16)
    
    print("Creating optimized datasets for 128-channel data...")
    train_dataset = OptimizedLandslideH5Dataset128(
        img_h5_paths=train_img_files,
        mask_h5_paths=train_mask_files,
        img_size=IMG_SIZE,
        max_samples=SUBSET_SIZE,
        is_train=True
    )
    
    # Split for validation (80/20)
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
    
    # Ultra-fast dataloaders
    train_loader = DataLoader(
        train_subset, 
        batch_size=BATCH_SIZE,
        shuffle=True, 
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2
    )
    
    val_loader = DataLoader(
        val_subset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=True
    )
    
    print(f"Training on {len(train_subset)} samples")
    print(f"Validating on {len(val_subset)} samples")
    print(f"Image size: {IMG_SIZE}x{IMG_SIZE}")
    print(f"Number of channels: 128")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Batches per epoch: {len(train_loader)}")
    
    # Model for 128 channels
    model = SegFormerLandslide128(num_classes=1, input_size=IMG_SIZE, output_size=IMG_SIZE)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Model parameters: {total_params:,}")
    
    # Memory estimation for 128 channels
    memory_per_sample = 128 * IMG_SIZE * IMG_SIZE * 4  # bytes for float32
    batch_memory = BATCH_SIZE * memory_per_sample / (1024**3)  # GB
    print(f"Estimated GPU memory per batch: {batch_memory:.2f} GB")
    
    print(f"\n🎯 128-Channel Training Summary:")
    print(f"   • Input shape: [B, 128, {IMG_SIZE}, {IMG_SIZE}]")
    print(f"   • Output shape: [B, 1, {IMG_SIZE}, {IMG_SIZE}]")
    print(f"   • Batch size: {BATCH_SIZE}")
    print(f"   • Estimated GPU memory: {batch_memory:.2f} GB")
    print(f"   • Model adapted for 128 input channels with upsampling")
    
    print("\nStarting ULTRA-FAST training for 128-channel data...")
    optimized_train_model_128(model, train_loader, val_loader, epochs=50, device=device)

# Run the modified training
if __name__ == "__main__":
    ultra_fast_training_128()

🚀 Starting ULTRA-FAST training for 128-channel data on RTX 4070...
Creating optimized datasets for 128-channel data...
Preloading file indices for 128-channel data...


Scanning HDF5 files:  10%|█         | 390/3799 [00:00<00:02, 1699.23it/s]


✓ Loaded 50000 samples with 128 channels
Training on 40000 samples
Validating on 10000 samples
Image size: 128x128
Number of channels: 128
Batch size: 16
Batches per epoch: 2500


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([1, 256, 1, 1]) in the model instantiated
- segformer.encoder.patch_embeddings.0.proj.weight: found shape torch.Size([32, 3, 7, 7]) in the checkpoint and torch.Size([32, 128, 7, 7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ SegFormer initialized for 128 input channels with upsampling
Model parameters: 3,910,401
Estimated GPU memory per batch: 0.12 GB

🎯 128-Channel Training Summary:
   • Input shape: [B, 128, 128, 128]
   • Output shape: [B, 1, 128, 128]
   • Batch size: 16
   • Estimated GPU memory: 0.12 GB
   • Model adapted for 128 input channels with upsampling

Starting ULTRA-FAST training for 128-channel data...
Starting optimized training for 128-channel data with mixed precision...


...


Epoch 48/50 (3.6 min):
  Train Loss: 0.2252, Train IoU: 0.3149
  Val Loss: 0.2208, Val IoU: 0.3232
  LR: 3.94e-07
  GPU Memory: 0.20GB


Epoch 49/50: 100%|██████████| 2500/2500 [02:58<00:00, 14.02it/s, loss=0.1998, avg_loss=0.2243, GPU_mem=0.20GB]



Epoch 49/50 (3.6 min):
  Train Loss: 0.2242, Train IoU: 0.3176
  Val Loss: 0.2209, Val IoU: 0.3220
  LR: 9.87e-08
  GPU Memory: 0.20GB


Epoch 50/50: 100%|██████████| 2500/2500 [02:56<00:00, 14.15it/s, loss=0.1898, avg_loss=0.2246, GPU_mem=0.20GB]



Epoch 50/50 (3.6 min):
  Train Loss: 0.2247, Train IoU: 0.3162
  Val Loss: 0.2205, Val IoU: 0.3241
  LR: 0.00e+00
  GPU Memory: 0.20GB
  ✓ Saved best model (val_iou: 0.3241)


In [6]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Load your best model ---
device = torch.device('cuda')
model = SegFormerLandslide128(num_classes=1) # Use your model class
model.load_state_dict(torch.load('best_landslide_model_128ch.pth'))
model.to(device)
model.eval() # Set model to evaluation mode

# --- 2. Get a sample from your validation set ---
# (Assuming 'val_loader' is your validation DataLoader)
sample_batch = next(iter(val_loader))
image_tensor = sample_batch['image'].to(device)
true_mask_tensor = sample_batch['mask'].to(device)

# --- 3. Make a prediction ---
with torch.no_grad():
    predicted_logits = model(image_tensor)
    predicted_mask = (torch.sigmoid(predicted_logits) > 0.5).float()

# --- 4. Visualize the first image in the batch ---
# Move tensors to CPU and convert to numpy
# Note: For visualizing a 128-channel image, we often just show an RGB composite (e.g., channels 3, 2, 1)
image_to_show = image_tensor[0].cpu().numpy()
rgb_composite = np.stack([image_to_show[3], image_to_show[2], image_to_show[1]], axis=-1)
# Normalize for display
rgb_composite = (rgb_composite - rgb_composite.min()) / (rgb_composite.max() - rgb_composite.min())


plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(rgb_composite)
plt.title("Input Image (RGB Composite)")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(true_mask_tensor[0].squeeze().cpu().numpy(), cmap='gray')
plt.title("Ground Truth")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(predicted_mask[0].squeeze().cpu().numpy(), cmap='gray')
plt.title(f"Model Prediction (IoU: {calculate_iou(predicted_logits[0], true_mask_tensor[0]):.4f})")
plt.axis('off')

plt.show()

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([1, 256, 1, 1]) in the model instantiated
- segformer.encoder.patch_embeddings.0.proj.weight: found shape torch.Size([32, 3, 7, 7]) in the checkpoint and torch.Size([32, 128, 7, 7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ SegFormer initialized for 128 input channels with upsampling


NameError: name 'val_loader' is not defined

In [1]:
import torch
import torch.nn as nn
from transformers import SegformerForSemanticSegmentation
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
from tqdm import tqdm
import h5py
from sklearn.model_selection import train_test_split
import cv2
import time

# ======================
# 1. SegFormer Model (Fixed)
# ======================

class SegFormerLandslide(nn.Module):
    def __init__(self, num_classes=1, input_size=512, output_size=512):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        # Load pre-trained SegFormer
        self.segformer = SegformerForSemanticSegmentation.from_pretrained(
            "nvidia/segformer-b0-finetuned-ade-512-512",
            num_labels=num_classes,
            ignore_mismatched_sizes=True
        )
        
        # Add upsampling layer to match output size
        self.upsample = nn.Upsample(size=output_size, mode='bilinear', align_corners=True)
        
        print("✓ SegFormer initialized with pre-trained weights + upsampling")
    
    def forward(self, x):
        outputs = self.segformer(x).logits  # [B, 1, 128, 128]
        return self.upsample(outputs)  # [B, 1, 512, 512]

# ======================
# 2. Loss Function
# ======================

class LandslideLoss(nn.Module):
    def __init__(self, alpha=0.7):
        super().__init__()
        self.alpha = alpha
        self.bce = nn.BCEWithLogitsLoss()
    
    def dice_loss(self, pred, target):
        smooth = 1.0
        pred = torch.sigmoid(pred)
        intersection = (pred * target).sum()
        union = pred.sum() + target.sum()
        return 1.0 - (2.0 * intersection + smooth) / (union + smooth)
    
    def forward(self, pred, target):
        bce_loss = self.bce(pred, target)
        dice_loss = self.dice_loss(pred, target)
        return self.alpha * bce_loss + (1 - self.alpha) * dice_loss

def calculate_iou(pred, target):
    pred = (torch.sigmoid(pred) > 0.5).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + 1e-6) / (union + 1e-6)

# ======================
# 3. OPTIMIZED Dataset Class for RTX 4070
# ======================

class OptimizedLandslideH5Dataset(Dataset):
    def __init__(self, img_h5_paths, mask_h5_paths=None, img_size=512, is_train=True, max_samples=None):
        self.img_h5_paths = img_h5_paths
        self.mask_h5_paths = mask_h5_paths
        self.img_size = img_size
        self.is_train = is_train
        
        # Optimized transform - minimal operations
        if is_train:
            self.transform = A.Compose([
                A.Resize(img_size, img_size),
                A.HorizontalFlip(p=0.3),
                A.VerticalFlip(p=0.3),
                ToTensorV2(),
            ], is_check_shapes=False)
        else:
            self.transform = A.Compose([
                A.Resize(img_size, img_size),
                ToTensorV2(),
            ], is_check_shapes=False)
        
        # PRELOAD all file indices for maximum speed
        self.total_samples = 0
        self.file_indices = []
        
        print("Preloading file indices for fast access...")
        for file_idx, img_path in enumerate(tqdm(img_h5_paths, desc="Scanning HDF5 files")):
            try:
                with h5py.File(img_path, 'r') as img_file:
                    img_key = list(img_file.keys())[0]
                    n_samples = img_file[img_key].shape[0]
                    
                    for sample_idx in range(n_samples):
                        if max_samples and self.total_samples >= max_samples:
                            break
                        self.file_indices.append((file_idx, sample_idx))
                        self.total_samples += 1
                    
                    if max_samples and self.total_samples >= max_samples:
                        break
            except Exception as e:
                continue
        
        print(f"✓ Loaded {self.total_samples} samples")
    
    def __len__(self):
        return self.total_samples
    
    def __getitem__(self, idx):
        file_idx, sample_idx = self.file_indices[idx]
        img_path = self.img_h5_paths[file_idx]
        mask_path = self.mask_h5_paths[file_idx] if self.mask_h5_paths else None
        
        try:
            # ULTRA-FAST loading - minimal error checking
            with h5py.File(img_path, 'r') as img_file:
                img_key = list(img_file.keys())[0]
                img = img_file[img_key][sample_idx]
                
                # Quick format conversion
                if len(img.shape) == 3 and img.shape[0] == 3:
                    img = np.transpose(img, (1, 2, 0))
                elif len(img.shape) == 2:
                    img = np.stack([img, img, img], axis=-1)
                
                # Take first 3 channels and normalize
                if img.shape[-1] > 3:
                    img = img[:, :, :3]
                img = img.astype(np.float32) / 255.0
                
                # Load mask
                mask = np.zeros(img.shape[:2], dtype=np.float32)
                if mask_path and os.path.exists(mask_path):
                    with h5py.File(mask_path, 'r') as mask_file:
                        mask_key = list(mask_file.keys())[0]
                        mask_data = mask_file[mask_key]
                        
                        if len(mask_data.shape) == 3 and sample_idx < mask_data.shape[0]:
                            mask = mask_data[sample_idx]
                        elif len(mask_data.shape) == 2:
                            mask = mask_data[:]
                        
                        if len(mask.shape) == 3:
                            mask = mask.squeeze()
                        mask = (mask > 0).astype(np.float32)
                
                # Apply transform
                transformed = self.transform(image=img, mask=mask)
                return {
                    'image': transformed['image'],
                    'mask': transformed['mask'].unsqueeze(0)
                }
                
        except Exception:
            # Fast fallback - zero tensor
            dummy_img = torch.zeros(3, self.img_size, self.img_size)
            dummy_mask = torch.zeros(1, self.img_size, self.img_size)
            return {'image': dummy_img, 'mask': dummy_mask}

# ======================
# 4. SPEED TEST Function
# ======================

def speed_test():
    """Comprehensive speed test for RTX 4070"""
    base_path = "/home/neel/Geog_project/Landslide4Sense_1"
    
    def find_h5_files(directory):
        if not os.path.exists(directory):
            return []
        return sorted([os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.h5')])
    
    train_img_files = find_h5_files(os.path.join(base_path, "TrainData", "img"))
    train_mask_files = find_h5_files(os.path.join(base_path, "TrainData", "mask"))
    
    print(f"Found {len(train_img_files)} training files")
    print(f"RTX 4070 detected - optimizing for high performance...")
    
    # Test configurations optimized for RTX 4070
    configs = [
        {"img_size": 512, "batch_size": 8, "max_samples": 2000, "num_workers": 6},
        {"img_size": 384, "batch_size": 12, "max_samples": 2000, "num_workers": 6},
        {"img_size": 256, "batch_size": 16, "max_samples": 2000, "num_workers": 6},
        {"img_size": 128, "batch_size": 32, "max_samples": 2000, "num_workers": 6},
    ]
    
    best_config = None
    best_speed = float('inf')
    
    for config in configs:
        print(f"\n{'='*50}")
        print(f"Testing config: {config}")
        print(f"{'='*50}")
        
        # Create dataset
        dataset = OptimizedLandslideH5Dataset(
            img_h5_paths=train_img_files[:20],  # More files for better test
            mask_h5_paths=train_mask_files[:20],
            img_size=config["img_size"],
            max_samples=config["max_samples"]
        )
        
        # Create optimized dataloader
        dataloader = DataLoader(
            dataset, 
            batch_size=config["batch_size"], 
            shuffle=True, 
            num_workers=config["num_workers"],
            pin_memory=True,
            persistent_workers=True  # Better for multiple epochs
        )
        
        # Initialize model
        model = SegFormerLandslide(num_classes=1, input_size=config["img_size"], output_size=config["img_size"])
        device = torch.device('cuda')
        model.to(device)
        
        # Warmup GPU
        print("Warming up GPU...")
        dummy_input = torch.randn(config["batch_size"], 3, config["img_size"], config["img_size"]).to(device)
        for _ in range(10):
            _ = model(dummy_input)
        
        # Time data loading
        print("Testing data loading speed...")
        data_times = []
        for i, batch in enumerate(dataloader):
            if i >= 20:  # Test 20 batches
                break
            start_time = time.time()
            images = batch['image'].to(device, non_blocking=True)
            masks = batch['mask'].to(device, non_blocking=True)
            data_times.append(time.time() - start_time)
        
        avg_data_time = np.mean(data_times)
        
        # Time forward/backward pass
        print("Testing training speed...")
        model.train()
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
        criterion = LandslideLoss()
        
        training_times = []
        for i, batch in enumerate(dataloader):
            if i >= 20:
                break
            images = batch['image'].to(device, non_blocking=True)
            masks = batch['mask'].to(device, non_blocking=True)
            
            start_time = time.time()
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            training_times.append(time.time() - start_time)
        
        avg_training_time = np.mean(training_times)
        total_batch_time = avg_data_time + avg_training_time
        
        # Calculate estimated epoch time
        samples_per_epoch = min(50000, len(dataset))  # Reasonable subset
        batches_per_epoch = samples_per_epoch // config["batch_size"]
        estimated_epoch_time = total_batch_time * batches_per_epoch / 60  # in minutes
        
        print(f"Results for {config['img_size']}x{config['img_size']}:")
        print(f"  Data loading: {avg_data_time*1000:.1f}ms per batch")
        print(f"  Training: {avg_training_time*1000:.1f}ms per batch")
        print(f"  Total: {total_batch_time*1000:.1f}ms per batch")
        print(f"  Estimated epoch: {estimated_epoch_time:.1f} minutes")
        print(f"  GPU Memory: {torch.cuda.memory_allocated()/1024**3:.1f}GB")
        
        if estimated_epoch_time < best_speed:
            best_speed = estimated_epoch_time
            best_config = config
    
    print(f"\n{'='*60}")
    print(f"🎯 BEST CONFIGURATION:")
    print(f"   Image Size: {best_config['img_size']}x{best_config['img_size']}")
    print(f"   Batch Size: {best_config['batch_size']}")
    print(f"   Estimated Epoch Time: {best_speed:.1f} minutes")
    print(f"{'='*60}")
    
    return best_config

# ======================
# 5. OPTIMIZED Training for RTX 4070
# ======================

def optimized_train_model(model, train_loader, val_loader, epochs=50, device='cuda'):
    model = model.to(device)
    
    # Mixed precision for RTX 4070
    scaler = torch.cuda.amp.GradScaler()
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    criterion = LandslideLoss(alpha=0.7)
    
    best_val_iou = 0.0
    
    print("Starting optimized training with mixed precision...")
    
    for epoch in range(epochs):
        epoch_start = time.time()
        
        # Training with mixed precision
        model.train()
        train_loss = 0
        train_iou = 0
        
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
        
        for batch_idx, batch in enumerate(train_bar):
            images = batch['image'].to(device, non_blocking=True)
            masks = batch['mask'].to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            # Mixed precision forward
            with torch.cuda.amp.autocast():
                outputs = model(images)
                if outputs.shape[-2:] != masks.shape[-2:]:
                    outputs = F.interpolate(outputs, size=masks.shape[-2:], mode='bilinear', align_corners=True)
                loss = criterion(outputs, masks)
            
            # Mixed precision backward
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            train_iou += calculate_iou(outputs, masks).item()
            
            if batch_idx % 50 == 0:
                train_bar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'avg_loss': f'{train_loss/(batch_idx+1):.4f}'
                })
        
        # Validation
        model.eval()
        val_loss = 0
        val_iou = 0
        
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device, non_blocking=True)
                masks = batch['mask'].to(device, non_blocking=True)
                
                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    if outputs.shape[-2:] != masks.shape[-2:]:
                        outputs = F.interpolate(outputs, size=masks.shape[-2:], mode='bilinear', align_corners=True)
                    loss = criterion(outputs, masks)
                
                val_loss += loss.item()
                val_iou += calculate_iou(outputs, masks).item()
        
        epoch_time = time.time() - epoch_start
        
        avg_train_loss = train_loss / len(train_loader)
        avg_train_iou = train_iou / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        avg_val_iou = val_iou / len(val_loader)
        
        scheduler.step()
        
        print(f'\nEpoch {epoch+1}/{epochs} ({epoch_time/60:.1f} min):')
        print(f'  Train Loss: {avg_train_loss:.4f}, Train IoU: {avg_train_iou:.4f}')
        print(f'  Val Loss: {avg_val_loss:.4f}, Val IoU: {avg_val_iou:.4f}')
        print(f'  LR: {scheduler.get_last_lr()[0]:.2e}')
        
        if avg_val_iou > best_val_iou:
            best_val_iou = avg_val_iou
            torch.save(model.state_dict(), 'best_landslide_model.pth')
            print(f'  ✓ Saved best model (val_iou: {avg_val_iou:.4f})')

# ======================
# 6. FAST TRAINING with Optimal Settings
# ======================
# ======================
# ULTRA-FAST TRAINING with Optimal Settings
# ======================

def ultra_fast_training():
    """Ultra-fast training with optimal settings from speed test"""
    device = torch.device('cuda')
    print(f"🚀 Starting ULTRA-FAST training on RTX 4070...")
    
    base_path = "/home/neel/Geog_project/Landslide4Sense_1"
    
    def find_h5_files(directory):
        if not os.path.exists(directory):
            return []
        return sorted([os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.h5')])
    
    train_img_files = find_h5_files(os.path.join(base_path, "TrainData", "img"))
    train_mask_files = find_h5_files(os.path.join(base_path, "TrainData", "mask"))
   
    IMG_SIZE = 128  # Better detail than 128
    BATCH_SIZE = 24  # Fits in 8GB VRAM with larger model
    SUBSET_SIZE = 40000  # Good amount of data
    NUM_WORKERS = min(os.cpu_count(), 24) # Use available cores, up to a max of 6
    
    print("Creating optimized datasets...")
    train_dataset = OptimizedLandslideH5Dataset(
        img_h5_paths=train_img_files,
        mask_h5_paths=train_mask_files,
        img_size=IMG_SIZE,
        max_samples=SUBSET_SIZE,
        is_train=True
    )
    
    # Split for validation (80/20)
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])
    
    # Ultra-fast dataloaders
    train_loader = DataLoader(
        train_subset, 
        batch_size=BATCH_SIZE,
        shuffle=True, 
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2
    )
    
    val_loader = DataLoader(
        val_subset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=True
    )
    
    print(f"Training on {len(train_subset)} samples")
    print(f"Validating on {len(val_subset)} samples")
    print(f"Image size: {IMG_SIZE}x{IMG_SIZE}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Batches per epoch: {len(train_loader)}")
    
    # Calculate expected epoch time
    expected_batch_time = 0.046  # From speed test (46.4ms)
    expected_epoch_time = expected_batch_time * len(train_loader)
    print(f"Expected epoch time: {expected_epoch_time:.1f} seconds ({expected_epoch_time/60:.1f} minutes)")
    
    # Model
    model = SegFormerLandslide(num_classes=1, input_size=IMG_SIZE, output_size=IMG_SIZE)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Model parameters: {total_params:,}")
    
    print(f"\n🎯 Training Summary:")
    print(f"   • Expected epoch time: {expected_epoch_time:.1f} seconds")
    print(f"   • Full training (50 epochs): {expected_epoch_time * 50 / 60:.1f} minutes")
    print(f"   • GPU Memory usage: ~0.6GB")
    print(f"   • Quality: Good for rapid prototyping")
    
    print("\nStarting ULTRA-FAST training...")
    optimized_train_model(model, train_loader, val_loader, epochs=50, device=device)

# Run ultra-fast training
ultra_fast_training()
# ======================
# MAIN EXECUTION
# ======================



🚀 Starting ULTRA-FAST training on RTX 4070...
Creating optimized datasets...
Preloading file indices for fast access...


Scanning HDF5 files:   8%|████▋                                                    | 312/3799 [00:00<00:00, 6145.76it/s]

✓ Loaded 40000 samples
Training on 32000 samples
Validating on 8000 samples
Image size: 128x128
Batch size: 24
Batches per epoch: 1334
Expected epoch time: 61.4 seconds (1.0 minutes)



Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([1, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ SegFormer initialized with pre-trained weights + upsampling
Model parameters: 3,714,401

🎯 Training Summary:
   • Expected epoch time: 61.4 seconds
   • Full training (50 epochs): 51.1 minutes
   • GPU Memory usage: ~0.6GB
   • Quality: Good for rapid prototyping

Starting ULTRA-FAST training...


...


Epoch 48/50 (1.3 min):
  Train Loss: 0.2153, Train IoU: 0.3353
  Val Loss: 0.2081, Val IoU: 0.3538
  LR: 3.94e-07


Epoch 49/50: 100%|████████████████████████████████████| 1334/1334 [01:12<00:00, 18.39it/s, loss=0.1657, avg_loss=0.2145]



Epoch 49/50 (1.3 min):
  Train Loss: 0.2146, Train IoU: 0.3369
  Val Loss: 0.2086, Val IoU: 0.3521
  LR: 9.87e-08


Epoch 50/50: 100%|████████████████████████████████████| 1334/1334 [01:15<00:00, 17.72it/s, loss=0.1760, avg_loss=0.2152]



Epoch 50/50 (1.4 min):
  Train Loss: 0.2153, Train IoU: 0.3356
  Val Loss: 0.2084, Val IoU: 0.3525
  LR: 0.00e+00


In [8]:
import joblib


# Example: Train a model

model = SegFormerLandslide()


# Define the filename
filename = '/home/neel/Geog_project/my_model.joblib'

# Export/Save the model to disk
joblib.dump(model, filename)




Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([1, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ SegFormer initialized with pre-trained weights + upsampling


['/home/neel/Geog_project/my_model.joblib']

In [11]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve, roc_curve, auc, confusion_matrix
import cv2
import os
from PIL import Image
import rasterio
from tqdm import tqdm
import pandas as pd

# ======================
# 1. COMPREHENSIVE EVALUATION METRICS
# ======================

class LandslideEvaluator:
    def __init__(self, model, device='cuda'):
        self.model = model
        self.device = device
        self.model.eval()
    
    def calculate_metrics(self, pred, target, threshold=0.5):
        """Calculate comprehensive segmentation metrics"""
        pred_binary = (pred > threshold).astype(np.float32)
        target_binary = (target > 0.5).astype(np.float32)
        
        # Basic metrics
        intersection = np.sum(pred_binary * target_binary)
        union = np.sum(pred_binary) + np.sum(target_binary) - intersection
        total_pixels = pred_binary.size
        
        # Avoid division by zero
        epsilon = 1e-6
        
        # Core metrics
        iou = intersection / (union + epsilon)
        dice = (2 * intersection) / (np.sum(pred_binary) + np.sum(target_binary) + epsilon)
        
        # Precision, Recall, F1
        true_positives = intersection
        false_positives = np.sum(pred_binary * (1 - target_binary))
        false_negatives = np.sum((1 - pred_binary) * target_binary)
        true_negatives = np.sum((1 - pred_binary) * (1 - target_binary))
        
        precision = true_positives / (true_positives + false_positives + epsilon)
        recall = true_positives / (true_positives + false_negatives + epsilon)
        f1_score = 2 * (precision * recall) / (precision + recall + epsilon)
        
        # Accuracy
        accuracy = (true_positives + true_negatives) / total_pixels
        
        # Specificity
        specificity = true_negatives / (true_negatives + false_positives + epsilon)
        
        # Balanced Accuracy
        balanced_accuracy = (recall + specificity) / 2
        
        return {
            'iou': iou,
            'dice': dice,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'accuracy': accuracy,
            'specificity': specificity,
            'balanced_accuracy': balanced_accuracy,
            'true_positives': true_positives,
            'false_positives': false_positives,
            'false_negatives': false_negatives,
            'true_negatives': true_negatives
        }
    
    def evaluate_single_image(self, image_path, mask_path=None, threshold=0.5):
        """Evaluate model on a single image"""
        # Load and preprocess image
        image = self.load_image(image_path)
        image_tensor = self.preprocess_image(image)
        
        # Predict
        with torch.no_grad():
            prediction = self.model(image_tensor.unsqueeze(0).to(self.device))
            prediction_prob = torch.sigmoid(prediction).squeeze().cpu().numpy()
        
        # Load ground truth if available
        ground_truth = None
        if mask_path and os.path.exists(mask_path):
            ground_truth = self.load_mask(mask_path)
        
        # Calculate metrics if ground truth exists
        metrics = None
        if ground_truth is not None:
            # Ensure same size
            if prediction_prob.shape != ground_truth.shape:
                ground_truth = cv2.resize(ground_truth, 
                                        (prediction_prob.shape[1], prediction_prob.shape[0]))
            
            metrics = self.calculate_metrics(prediction_prob, ground_truth, threshold)
        
        return {
            'image': image,
            'prediction': prediction_prob,
            'ground_truth': ground_truth,
            'metrics': metrics,
            'binary_prediction': (prediction_prob > threshold).astype(np.float32)
        }
    
    def evaluate_dataset(self, image_dir, mask_dir=None, threshold=0.5):
        """Evaluate model on entire dataset"""
        image_files = [f for f in os.listdir(image_dir) if f.endswith(('.tif', '.png', '.jpg', '.jpeg'))]
        
        all_metrics = []
        results = []
        
        print(f"Evaluating on {len(image_files)} images...")
        
        for image_file in tqdm(image_files):
            image_path = os.path.join(image_dir, image_file)
            
            # Find corresponding mask
            mask_path = None
            if mask_dir:
                mask_candidates = [
                    image_file.replace('.tif', '.png').replace('.jpg', '.png').replace('.jpeg', '.png'),
                    image_file,
                    image_file.replace('.png', '.tif')
                ]
                
                for mask_candidate in mask_candidates:
                    candidate_path = os.path.join(mask_dir, mask_candidate)
                    if os.path.exists(candidate_path):
                        mask_path = candidate_path
                        break
            
            try:
                result = self.evaluate_single_image(image_path, mask_path, threshold)
                
                if result['metrics'] is not None:
                    all_metrics.append(result['metrics'])
                
                results.append({
                    'image_file': image_file,
                    'metrics': result['metrics'],
                    'has_ground_truth': mask_path is not None
                })
                
            except Exception as e:
                print(f"Error processing {image_file}: {e}")
                continue
        
        return results, all_metrics
    
    def load_image(self, image_path):
        """Load image in various formats"""
        if image_path.endswith('.tif'):
            with rasterio.open(image_path) as src:
                image = src.read()[:3]  # Take first 3 bands
                image = np.transpose(image, (1, 2, 0))
        else:
            image = np.array(Image.open(image_path).convert('RGB'))
        
        return image
    
    def load_mask(self, mask_path):
        """Load mask in various formats"""
        if mask_path.endswith('.tif'):
            with rasterio.open(mask_path) as src:
                mask = src.read(1)
        else:
            mask = np.array(Image.open(mask_path).convert('L'))
        
        # Binarize mask
        mask = (mask > 0).astype(np.float32)
        return mask
    
    def preprocess_image(self, image):
        """Preprocess image for model input"""
        # Resize to model input size
        if hasattr(self.model, 'input_size'):
            target_size = self.model.input_size
        else:
            target_size = 256  # Default
        
        image_resized = cv2.resize(image, (target_size, target_size))
        
        # Normalize
        image_normalized = image_resized.astype(np.float32) / 255.0
        
        # Convert to tensor
        image_tensor = torch.from_numpy(image_normalized).permute(2, 0, 1).float()
        
        return image_tensor

# ======================
# 2. VISUALIZATION TOOLS
# ======================

class ResultVisualizer:
    def __init__(self):
        self.cmap = plt.cm.viridis
    
    def plot_single_result(self, result, threshold=0.5, save_path=None):
        """Plot single image result with prediction"""
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.ravel()
        
        image = result['image']
        prediction = result['prediction']
        ground_truth = result['ground_truth']
        binary_pred = result['binary_prediction']
        metrics = result['metrics']
        
        # Original image
        axes[0].imshow(image)
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        # Prediction probability
        im1 = axes[1].imshow(prediction, cmap='viridis', vmin=0, vmax=1)
        axes[1].set_title('Prediction Probability')
        axes[1].axis('off')
        plt.colorbar(im1, ax=axes[1])
        
        # Binary prediction
        axes[2].imshow(binary_pred, cmap='gray')
        axes[2].set_title(f'Binary Prediction (threshold={threshold})')
        axes[2].axis('off')
        
        # Ground truth (if available)
        if ground_truth is not None:
            axes[3].imshow(ground_truth, cmap='gray')
            axes[3].set_title('Ground Truth')
            axes[3].axis('off')
            
            # Overlay prediction on image
            axes[4].imshow(image)
            axes[4].imshow(binary_pred, alpha=0.5, cmap='Reds')
            axes[4].set_title('Prediction Overlay (Red)')
            axes[4].axis('off')
            
            # Difference map
            difference = np.abs(binary_pred - ground_truth)
            axes[5].imshow(difference, cmap='coolwarm')
            axes[5].set_title('Difference Map\n(Red=False Pos, Blue=False Neg)')
            axes[5].axis('off')
            
            # Add metrics text
            if metrics:
                metrics_text = f"IoU: {metrics['iou']:.3f}\nDice: {metrics['dice']:.3f}\nF1: {metrics['f1_score']:.3f}\nPrecision: {metrics['precision']:.3f}\nRecall: {metrics['recall']:.3f}"
                fig.text(0.02, 0.5, metrics_text, fontsize=12, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray"))
        
        else:
            # If no ground truth, show different visualizations
            axes[3].axis('off')
            
            # Overlay probability on image
            axes[4].imshow(image)
            axes[4].imshow(prediction, alpha=0.6, cmap='viridis')
            axes[4].set_title('Probability Overlay')
            axes[4].axis('off')
            
            # Confidence histogram
            axes[5].hist(prediction.flatten(), bins=50, alpha=0.7)
            axes[5].set_title('Prediction Confidence Distribution')
            axes[5].set_xlabel('Confidence')
            axes[5].set_ylabel('Frequency')
            axes[5].axvline(threshold, color='red', linestyle='--', label=f'Threshold ({threshold})')
            axes[5].legend()
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"Saved visualization to {save_path}")
        
        plt.show()
    
    def plot_metrics_distribution(self, all_metrics, save_path=None):
        """Plot distribution of metrics across dataset"""
        if not all_metrics:
            print("No metrics to plot")
            return
        
        metrics_df = pd.DataFrame(all_metrics)
        
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.ravel()
        
        # Plot distributions
        metrics_to_plot = ['iou', 'dice', 'f1_score', 'precision', 'recall', 'accuracy']
        titles = ['IoU Score', 'Dice Coefficient', 'F1 Score', 'Precision', 'Recall', 'Accuracy']
        
        for i, (metric, title) in enumerate(zip(metrics_to_plot, titles)):
            axes[i].hist(metrics_df[metric], bins=20, alpha=0.7, edgecolor='black')
            axes[i].set_title(f'{title} Distribution')
            axes[i].set_xlabel(title)
            axes[i].set_ylabel('Frequency')
            axes[i].axvline(metrics_df[metric].mean(), color='red', linestyle='--', 
                          label=f'Mean: {metrics_df[metric].mean():.3f}')
            axes[i].legend()
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        
        plt.show()
        
        return metrics_df
    
    def plot_confusion_matrix(self, all_metrics, save_path=None):
        """Plot confusion matrix from aggregated metrics"""
        if not all_metrics:
            print("No metrics for confusion matrix")
            return
        
        total_tp = sum(m['true_positives'] for m in all_metrics)
        total_fp = sum(m['false_positives'] for m in all_metrics)
        total_fn = sum(m['false_negatives'] for m in all_metrics)
        total_tn = sum(m['true_negatives'] for m in all_metrics)
        
        cm = np.array([[total_tn, total_fp], 
                      [total_fn, total_tp]])
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='.0f', cmap='Blues',
                   xticklabels=['Predicted Negative', 'Predicted Positive'],
                   yticklabels=['Actual Negative', 'Actual Positive'])
        plt.title('Confusion Matrix (Aggregated)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        
        plt.show()

# ======================
# 3. THRESHOLD OPTIMIZATION
# ======================

def find_optimal_threshold(all_metrics, all_predictions, all_ground_truths):
    """Find optimal threshold using precision-recall curve"""
    # Flatten all predictions and ground truths
    all_pred_flat = np.concatenate([pred.flatten() for pred in all_predictions])
    all_gt_flat = np.concatenate([gt.flatten() for gt in all_ground_truths])
    
    # Calculate precision-recall curve
    precision, recall, thresholds = precision_recall_curve(all_gt_flat, all_pred_flat)
    
    # Calculate F1 score for each threshold
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-6)
    
    # Find optimal threshold (max F1)
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx]
    optimal_f1 = f1_scores[optimal_idx]
    
    # Plot PR curve
    plt.figure(figsize=(10, 6))
    plt.plot(recall, precision, linewidth=2, label='Precision-Recall Curve')
    plt.plot(recall[optimal_idx], precision[optimal_idx], 'ro', 
             label=f'Optimal (F1={optimal_f1:.3f}, threshold={optimal_threshold:.3f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return optimal_threshold, optimal_f1

# ======================
# 4. COMPLETE EVALUATION PIPELINE
# ======================

def comprehensive_evaluation(model_path, test_image_dir, test_mask_dir=None, output_dir='evaluation_results'):
    """Complete evaluation pipeline"""
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Load model
    print("Loading trained model...")
    model = load_trained_model(model_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Initialize evaluator and visualizer
    evaluator = LandslideEvaluator(model, device)
    visualizer = ResultVisualizer()
    
    # Evaluate dataset
    print("Evaluating on test dataset...")
    results, all_metrics = evaluator.evaluate_dataset(test_image_dir, test_mask_dir)
    
    # Save results
    results_df = pd.DataFrame([r for r in results if r['metrics'] is not None])
    results_df.to_csv(os.path.join(output_dir, 'evaluation_results.csv'), index=False)
    
    # Generate comprehensive report
    if all_metrics:
        metrics_df = visualizer.plot_metrics_distribution(all_metrics, 
                                                         os.path.join(output_dir, 'metrics_distribution.png'))
        
        visualizer.plot_confusion_matrix(all_metrics, 
                                       os.path.join(output_dir, 'confusion_matrix.png'))
        
        # Calculate overall statistics
        overall_stats = {
            'mean_iou': np.mean([m['iou'] for m in all_metrics]),
            'mean_dice': np.mean([m['dice'] for m in all_metrics]),
            'mean_f1': np.mean([m['f1_score'] for m in all_metrics]),
            'mean_precision': np.mean([m['precision'] for m in all_metrics]),
            'mean_recall': np.mean([m['recall'] for m in all_metrics]),
            'mean_accuracy': np.mean([m['accuracy'] for m in all_metrics]),
            'std_iou': np.std([m['iou'] for m in all_metrics]),
            'num_samples': len(all_metrics)
        }
        
        # Save overall statistics
        stats_df = pd.DataFrame([overall_stats])
        stats_df.to_csv(os.path.join(output_dir, 'overall_statistics.csv'), index=False)
        
        print("\n📊 OVERALL PERFORMANCE SUMMARY:")
        print(f"   • Number of samples: {overall_stats['num_samples']}")
        print(f"   • Mean IoU: {overall_stats['mean_iou']:.4f} ± {overall_stats['std_iou']:.4f}")
        print(f"   • Mean Dice: {overall_stats['mean_dice']:.4f}")
        print(f"   • Mean F1: {overall_stats['mean_f1']:.4f}")
        print(f"   • Mean Precision: {overall_stats['mean_precision']:.4f}")
        print(f"   • Mean Recall: {overall_stats['mean_recall']:.4f}")
        print(f"   • Mean Accuracy: {overall_stats['mean_accuracy']:.4f}")
    
    # Visualize some sample results
    print("\nGenerating sample visualizations...")
    sample_results = [r for r in results if r['has_ground_truth']][:5]  # First 5 with ground truth
    
    for i, result_info in enumerate(sample_results):
        image_path = os.path.join(test_image_dir, result_info['image_file'])
        mask_path = os.path.join(test_mask_dir, result_info['image_file']) if test_mask_dir else None
        
        result = evaluator.evaluate_single_image(image_path, mask_path)
        visualizer.plot_single_result(
            result, 
            save_path=os.path.join(output_dir, f'sample_result_{i+1}.png')
        )
    
    print(f"\n✅ Evaluation complete! Results saved to: {output_dir}")
    return results, all_metrics

# ======================
# 5. QUICK SINGLE IMAGE TEST
# ======================

def test_single_image(model_path, image_path, mask_path=None, threshold=0.5):
    """Quick test on a single image"""
    
    # Load model
    model = load_trained_model(model_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Initialize evaluator and visualizer
    evaluator = LandslideEvaluator(model, device)
    visualizer = ResultVisualizer()
    
    # Evaluate single image
    result = evaluator.evaluate_single_image(image_path, mask_path, threshold)
    
    # Visualize results
    visualizer.plot_single_result(result, threshold)
    
    # Print metrics if available
    if result['metrics']:
        print("\n📈 PERFORMANCE METRICS:")
        for metric, value in result['metrics'].items():
            if isinstance(value, (int, float)):
                print(f"   • {metric.replace('_', ' ').title()}: {value:.4f}")
    
    return result

# ======================
# 6. USAGE EXAMPLES
# ======================

if __name__ == "__main__":
    
    # Example 1: Test single image (no ground truth needed)
    print("=== SINGLE IMAGE TEST ===")
    result = test_single_image(
        model_path='/home/neel/Geog_project/best_landslide_model.pth',
        image_path='/home/neel/Geog_project/myanmar_ali_2015296.jpg',
        mask_path=None,  # Optional: provide if you have ground truth
        threshold=0.5
    )
    
    # Example 2: Comprehensive evaluation on dataset
    print("\n=== COMPREHENSIVE DATASET EVALUATION ===")
    results, metrics = comprehensive_evaluation(
        model_path='/home/neel/Geog_project/my_model.joblib',
        test_image_dir='/path/to/your/test/images',
        test_mask_dir='/path/to/your/test/masks',  # Optional
        output_dir='evaluation_results'
    )
    
    # Example 3: Batch process multiple images without ground truth
    print("\n=== BATCH PREDICTION (NO GROUND TRUTH) ===")
    model = load_trained_model('best_landslide_model_improved.pth')
    evaluator = LandslideEvaluator(model)
    
    image_dir = 'Geog_project/myanmar_ali_2015296.jpg'
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.tif', '.png', '.jpg'))]
    
    for image_file in image_files[:3]:  # Test first 3 images
        image_path = os.path.join(image_dir, image_file)
        print(f"\nProcessing: {image_file}")
        
        result = evaluator.evaluate_single_image(image_path)
        
        # Save prediction
        pred_image = (result['prediction'] * 255).astype(np.uint8)
        output_path = os.path.join('predictions', f'pred_{image_file}')
        os.makedirs('predictions', exist_ok=True)
        Image.fromarray(pred_image).save(output_path)
        print(f"Saved prediction to: {output_path}")

=== SINGLE IMAGE TEST ===


NameError: name 'load_trained_model' is not defined